## <u>Classify Fruits and Vegetables</u>
### Dataset: Fruits360

- Then to build an iOS app to use the machine learning model.

In [95]:
# importing
import random
import pandas as pd
import numpy as np
import torchvision
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import cv2
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from PIL import Image
import os
from io import open
from torchvision.transforms import transforms
from torchvision.models import resnet18

In [96]:
# Loading data
transforms_train = T.Compose([T.ToTensor(),T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])
image_data_train = ImageFolder("./Fruit-Images-Dataset/data",transform=transforms_train)
image_data_test = ImageFolder("./Fruit-Images-Dataset/Test2",transform=transforms_train)



# Shuffling data and then collecting all the labels.
random.shuffle(image_data_train.samples)
random.shuffle(image_data_test.samples)

In [97]:
# Total classes
classes_idx = image_data_train.class_to_idx
classes = len(image_data_train.classes)
len_train_data = len(image_data_train)
len_test_data = len(image_data_test)



def get_labels():
    labels_train = [] # All the labels
    labels_test = []
    for i in image_data_train.imgs:
        labels_train.append(i[1])
    
    for j in image_data_test.imgs:
        labels_test.append(j[1])
    
    return (labels_train, labels_test)

labels_train, labels_test = get_labels()


In [98]:
train_loader = DataLoader(dataset=image_data_train,batch_size=100)
test_loader = DataLoader(dataset=image_data_test,batch_size=100)

In [99]:
print (iter(train_loader).next()[0].shape)
record = iter(train_loader).next()[0]

torch.Size([100, 3, 100, 100])


- We can see that the image is (batch_size, channel, image_height, image_width)

In [100]:
# Flatten Layer
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
# Main Model
class Model:
    def build_model(self):
        model = nn.Sequential(nn.Conv2d(3, 64, kernel_size=5, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Conv2d(64, 64, kernel_size=7, stride=1),
                              nn.ReLU(),
                              nn.MaxPool2d(3),
                              nn.Conv2d(64, 64, kernel_size=7),
                              nn.ReLU(),
                              nn.MaxPool2d(5),
                              Flatten(),
                              nn.Linear(64, 100),
                              nn.ReLU(),
                              nn.Linear(100, 65))
        return model

In [101]:
# Building 
model = Model()
model = model.build_model()

# Checking for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=65, bias=True)
)

In [102]:
print (model)
print (labels_train[0])
print (image_data_train.samples[0][1])

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=65, bias=True)
)
14
14


In [103]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [104]:
# Train the model
def train(epochs):
    model.train()
    losses = []
    for epoch in range(1, epochs+1):
        print ("epoch #", epoch)
        current_loss = 0.0
        for feature, label in train_loader:
            x = Variable(feature, requires_grad=False).float().to(device)
            y = Variable(label, requires_grad=False).long().to(device)
            optimizer.zero_grad() # Zeroing the grads
            y_pred = model(x) # Calculating prediction
            correct = y_pred.max(1)[1].eq(y).sum()
            print ("no. of correct items classified: ", correct.item())
            loss = criterion(y_pred, y) # Calculating loss (log_softmax already included)
            print ("loss: ", loss.item())
            current_loss+=loss.item()
            loss.backward() # Gradient cal
            optimizer.step() # Changing weights
        losses.append(current_loss) # Only storing loss after every epoch
    return losses
# Test the model
def test():
    model.eval()
    with torch.no_grad():
        for feature, label in test_loader:
            pred = model(feature)
            print ("acc: ", accuracy_score(labels_test, pred.max(1)[1].data.numpy()) * 100)
            loss = criterion(pred, label)
            print ("loss: ", loss.item())

In [105]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=65, bias=True)
)

In [106]:
load_state_dict = torch.load("./model-state-dict.pth",map_location="cpu")
# model = resnet18(pretrained=True)

In [107]:
model.load_state_dict(load_state_dict)

<All keys matched successfully>

In [108]:
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten()
  (10): Linear(in_features=64, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=65, bias=True)
)

In [109]:
def predict_image(imagefile):
    print("Prediction in progress")
    
    imagepath = os.path.join(os.getcwd(), imagefile)
    image = Image.open(imagepath)

    # Define transformations for the image, should (note that imagenet models are trained with image size 224)
    transformation = transforms.Compose([
#         transforms.CenterCrop(100),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

    ])

    # Preprocess the image
    image_tensor = transformation(image).float()

    # Add an extra batch dimension since pytorch treats all images as batches
    image_tensor = image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    # Turn the input into a Variable
    input = Variable(image_tensor)

    # Predict the class of the image
    output = model(input)

    index = output.data.numpy().argmax()

    return index

In [118]:
abc = predict_image("Tomato2.jpg")

Prediction in progress


In [119]:
print(abc)

44


In [81]:
print(classes_idx)

{'Apple Braeburn': 0, 'Apple Crimson Snow': 1, 'Apple Golden 1': 2, 'Apple Golden 2': 3, 'Apple Golden 3': 4, 'Apple Granny Smith': 5, 'Apple Pink Lady': 6, 'Apple Red 1': 7, 'Apple Red 2': 8, 'Apple Red 3': 9, 'Apple Red Delicious': 10, 'Apple Red Yellow 1': 11, 'Apple Red Yellow 2': 12, 'Apricot': 13, 'Avocado': 14, 'Avocado ripe': 15, 'Banana': 16, 'Banana Lady Finger': 17, 'Banana Red': 18, 'Beetroot': 19, 'Blueberry': 20, 'Cactus fruit': 21, 'Cantaloupe 1': 22, 'Cantaloupe 2': 23, 'Carambula': 24, 'Cauliflower': 25, 'Cherry 1': 26, 'Cherry 2': 27, 'Cherry Rainier': 28, 'Cherry Wax Black': 29, 'Cherry Wax Red': 30, 'Cherry Wax Yellow': 31, 'Chestnut': 32, 'Clementine': 33, 'Cocos': 34, 'Dates': 35, 'Eggplant': 36, 'Ginger Root': 37, 'Granadilla': 38, 'Grape Blue': 39, 'Grape Pink': 40, 'Grape White': 41, 'Grape White 2': 42, 'Grape White 3': 43, 'Grape White 4': 44, 'Grapefruit Pink': 45, 'Grapefruit White': 46, 'Guava': 47, 'Hazelnut': 48, 'Huckleberry': 49, 'Kaki': 50, 'Kiwi': 51